In [1]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import model_from_json
# from tensorflow.keras import np_utils
from tensorflow.keras.optimizers import Adam
import tensorflow.keras.callbacks
# import tensorflow.keras.backend.tensorflow_backend as KTF

import os.path
from sklearn.datasets import load_files
import pandas as pd
from tqdm import tqdm#progress bar
from PIL import Image

from datetime import datetime as dt

tqdm.pandas()#progress bar

In [2]:
def apply_loop(frompd,arg1):
    arg1.append(np.asarray(Image.open(frompd).convert('RGB')))

def load_dataset(path):
    data=load_files(path)
    targets_img = []

    data_df = pd.DataFrame(columns=['filenames', 'target_names'])
    data_df['filenames'] = data['filenames']
    data_df['target'] = data['target']
    data_df['img'] = data_df['filenames'].progress_apply(apply_loop,arg1=targets_img)

    # 返り値にそれぞれ　画像のリスト、エンコーディングされた整数値、カテゴリーリスト
    return targets_img, data_df['target'] ,data["target_names"]


In [3]:
def saveimgloop(frompd,arg1,dirstr):
    # df = pd.DataFrame(columns=['img_num', 'ans', 'pred'])
    Image.fromarray(np.uint8(arg1["data"][frompd["img_num"]]*255)).save("{0}/missimg{1}_ans_{2}_predict{3}.png".format(dirstr,frompd["img_num"],arg1["labels_list"][frompd["ans"]],arg1["labels_list"][frompd["pred"]]))
    

In [4]:
batch_size = 256
nb_classes = 120
nb_epoch = 1
epochs = 20

dirstr = "./miss_img/batch_size{0}_epochs{1}_{2}".format(batch_size,epochs,dt.now().strftime("%Y_%m_%d_%H_%M_%S"))

f_log = './log'
f_model = './model'
model_filename = './miss_img/batch_size256_epochs20_2020_07_30_01_40_07/vgg16_model.json'
weights_filename = './miss_img/batch_size256_epochs20_2020_07_30_01_40_07/vgg16_weights.hdf5'

# data load---------------
traindata = np.load('fruits_train.npz')
testdata = np.load('fruits_test.npz')


In [5]:
with tf.Graph().as_default():

    json_string = open(os.path.join(model_filename)).read()
    model = model_from_json(json_string)

    model.summary()

    model.compile(loss='categorical_crossentropy', optimizer=Adam(lr=0.001, beta_1=0.5), metrics=['accuracy'])

    model.load_weights(os.path.join(weights_filename))

    cbks = []

    # score = model.evaluate(testdata['data'], testdata['target_labels'], verbose=1)
    predictdata = model.predict(testdata['data'],verbose=1)

    # print('Test score:', score[0])
    # print('Test accuracy:', score[1])


Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_14 (Conv2D)           (None, 100, 100, 8)       224       
_________________________________________________________________
batch_normalization_14 (Batc (None, 100, 100, 8)       32        
_________________________________________________________________
activation_14 (Activation)   (None, 100, 100, 8)       0         
_________________________________________________________________
conv2d_15 (Conv2D)           (None, 100, 100, 64)      4672      
_________________________________________________________________
batch_normalization_15 (Batc (None, 100, 100, 64)      256       
_________________________________________________________________
activation_15 (Activation)   (None, 100, 100, 64)      0         
_______________________________________

In [6]:
    print("抽出")
    # 不正解画像番号抽出
    miss_data_nums = []
    """
    for cnt,predata in tqdm(enumerate(predictdata)):
        if np.where(predata == predata.max()) != np.where(testdata["target_labels"][cnt] == testdata["target_labels"][cnt].max()):
            miss_data_nums.append([cnt,np.where(testdata["target_labels"][cnt] == testdata["target_labels"][cnt].max()),np.where(predata == predata.max())])
    """
    predictdata_max = np.argmax(predictdata,axis=1)
    test_target_labels = np.argmax(testdata["target_labels"],axis=1)
    for cnt,predata in tqdm(enumerate(predictdata_max)):
        if predata != test_target_labels[cnt]:
            miss_data_nums.append([cnt,test_target_labels[cnt],predata])
    print("抽出終了")
    print("画像保存")
    
    df = pd.DataFrame(columns=['img_num', 'ans', 'pred'])
    df["img_num"] = miss_data_nums[0]
    df["ans"] = miss_data_nums[1]
    df["pred"] = miss_data_nums[2]

    os.makedirs(dirstr)
    # df.progress_apply(saveimgloop,arg1=testdata,dirstr=dirstr)

    with open('{}/err_detail.log'.format(dirstr), mode='w') as f:
        f.write("Fruits_list,")
        for labelist in testdata["labels_list"]:
            f.write("{},".format(labelist))
        f.write("\n")
        for num in miss_data_nums:
            f.write("{},".format(testdata["labels_list"][predictdata_max[num[0]]]))
            for predictlist in predictdata[num[0]]:
                f.write("{},".format(predictlist))
            f.write("\n")
            


    # """
    for num in tqdm(miss_data_nums):
        # print(x_test_raw[num])
        # Image.fromarray(np.uint8(traindata["data"][0]*255)).show()
        Image.fromarray(np.uint8(testdata["data"][num[0]]*255)).save("{0}/missimg{1}_ans_{2}_predict{3}.png".format(dirstr,num[0],testdata["labels_list"][num[1]],testdata["labels_list"][num[2]]))
    # """
    
    

抽出
20622it [00:00, 2072628.61it/s]抽出終了
画像保存



'\nfor num in tqdm(miss_data_nums):\n    # print(x_test_raw[num])\n    # Image.fromarray(np.uint8(traindata["data"][0]*255)).show()\n    Image.fromarray(np.uint8(testdata["data"][num[0]]*255)).save("{0}/missimg{1}_ans_{2}_predict{3}.png".format(dirstr,num[0],testdata["labels_list"][num[1]],testdata["labels_list"][num[2]]))\n'